In [2]:
# Importa bilbiotecas
import numpy as np
import math as m
import time as t

In [3]:
# Lê as coordenadas dos vértices
def leVertices(arquivo):
  coords = []
  line = ' '
  flag = False;
  while line:
      line = arquivo.readline();
      split = line.split()
      if "EOF" in line:
        flag = False;
      if flag:
        coord = (int(split[0]), float(split[1]), float(split[2]))
        coords.append(coord)
      if "DISPLAY_DATA_SECTION" in line or "NODE_COORD_SECTION" in line:
        flag = True;
  return coords

In [4]:
# Calcula distância euclidiana entre dois pontos
def calculaDistancia(a, b):
    return float("{:.2f}".format((m.sqrt(((b[1]-a[1])**2) + ((b[2]-a[2])**2)))))

In [5]:
# Métodos de criação e acesso à matriz de distância 
def montaMatrizDistancia(coords):
  size = len(coords)
  dist = []
  for i in range(size):
    row = []
    for j in range(i+1):
      row.append(calculaDistancia(coords[i], coords[j]))
    dist.append(row)
  
  return dist

def acessa(dist, i, j):
  if i >= j: return dist[i][j]
  else: return dist[j][i]

In [6]:
# Define o percurso através da heurística do Vizinho Mais Próximo
# Complexidade: O(n²)

def maisProximo(dist, i, nao_visitados):
  menor = nao_visitados[0]
  for j in nao_visitados:
    if acessa(dist, i, j) < acessa(dist, i, menor):
      menor = j

  return menor

def vizinhoMasProximo(dist):
  visitados = []
  visitados.append(0)
  nao_visitados = []
  for i in range(1, len(dist)):
    nao_visitados.append(i)

  while nao_visitados:
    maisProx = maisProximo(dist, visitados[-1], nao_visitados)
    nao_visitados.remove(maisProx)
    visitados.append(maisProx)

  visitados.append(0)
  return visitados

In [7]:
# Melhora o percurso obtido pelo VMP utilizando o 2-opt
def doisOpt(seqVert, rota, dist):
  arestas = rota[:]
  maiorGanho = 0;
  swap = ()
  for i in range(len(arestas)):
    for j in range(len(arestas)):
      if arestas[i][0] != arestas[j][1] and arestas[i][1] != arestas[j][0] and arestas[i][0] != arestas[j][0]:
        pesoAB = arestas[i][2]
        pesoXY = arestas[j][2]
        pesoAX = acessa(dist, arestas[i][0], arestas[j][0])
        pesoBY = acessa(dist, arestas[i][1], arestas[j][1])
        pesoAtual = pesoAB + pesoXY
        pesoAlt = pesoAX + pesoBY
        if pesoAtual - pesoAlt > maiorGanho:
          maiorGanho = pesoAtual - pesoAlt
          swap = (i, j)

  if maiorGanho > 0:
    i = min(swap[0], swap[1])
    j = max(swap[0], swap[1])
    indiceIni = 0
    indiceFim = 0
    for k in range(len(seqVert)):
      if(seqVert[k] == arestas[i][1]):
        indiceIni = k
      if(seqVert[k] == arestas[j][0]):
        indiceFim = k

    novaSeq = seqVert[:]
    for i in range(len(seqVert)):
      if(i >= indiceIni and i <= indiceFim):
        novaSeq[i] = seqVert[indiceFim-(i-indiceIni)]

  return novaSeq

In [8]:
# Monta lista de arestas da uma rota
def montaListaArestas(rota, dist):
  arestas = []
  for i in range(len(rota) - 1):
    aresta = []
    aresta.append(rota[i])
    aresta.append(rota[i+1])
    aresta.append(acessa(dist, rota[i], rota[i+1]))
    arestas.append(aresta)
  
  return arestas

In [9]:
# Calcula a distância total de uma rota
def distanciaTotal(arestas):
  distancia = 0.0
  for a in arestas:
    distancia += a[2]
  
  return float("{:.2f}".format(distancia))

In [10]:
# Executa algoritmos de construção e melhoria da solução para o PCV
def processaDados(dist):
  seqVert = vizinhoMasProximo(dist)
  rota = montaListaArestas(seqVert, dist)
  seqVertOtimizada = doisOpt(seqVert, rota, dist)
    
  return seqVertOtimizada

In [26]:
def executaBaseDeDados():
  casosTeste = [ 'ch130', 'd1291', 'd1655', 'd2103',
                'd18512', 'fl1577', 'pr439', 'rd400',
                'ts225', 'u2319', 'ulysses16', 'ulysses22']

  for ct in casosTeste:
    nomeArquivo = ct
    arquivo = open("sample_data/" + nomeArquivo + ".tsp", "r")

    coords = leVertices(arquivo)
    dist = montaMatrizDistancia(coords)

    inicioExec = t.time()
    seqFinal = processaDados(dist)
    fimExec = t.time()

    disTotal = distanciaTotal(montaListaArestas(seqFinal, dist))
    salvaResultados(nomeArquivo, round((fimExec - inicioExec), 4), disTotal, seqFinal)



In [14]:
def salvaResultados(nomeArquivo, tempoExec, disTotal, seqFinal):
  arquivo = open("sample_data/" + nomeArquivo + "_result.txt", "w+")
  arquivo.write("TEMPO_EXEC " + str(tempoExec))
  arquivo.write("\nDISTANCIA_TOTAL " + str(disTotal))
  arquivo.write("\nSEQUENCIA_DE_VISITACAO")
  for i in range(len(seqFinal)):
    arquivo.write("\n\t" + str(i+1) + "  " + str(seqFinal[i]))
  arquivo.write("\nEOF")
  arquivo.close()

In [ ]:
# Executa o algoritmo para o arquivo 

# -------------------------------------- IMPORTANTE ---------------------------------------
# Para executar o código corretamente, por favor, insira os arquivos na pasta 'sample_data'
# -----------------------------------------------------------------------------------------

print("Digite o nome do arquivo de entrada .tsp:")
print("(Ex.: ts225)")
nomeArquivo = input()
arquivo = open("sample_data/" + nomeArquivo + ".tsp", "r")

coords = leVertices(arquivo)
dist = montaMatrizDistancia(coords)

inicioExec = t.time()
seqFinal = processaDados(dist)
fimExec = t.time()

disTotal = distanciaTotal(montaListaArestas(seqFinal, dist))

print("\nTempo total de execução (em segundos):", round((fimExec - inicioExec), 4))
print("Distância total do percurso encontrado:", disTotal, "metros.\n")
print("Sequência de pontos de coleta de lixo a serem visitados:")
for i in range(len(seqFinal)): print(i+1, "º ponto:", seqFinal[i])

salvaResultados(nomeArquivo, round((fimExec - inicioExec), 4), disTotal, seqFinal)


In [21]:
# Descomente para executar o algoritmo para a base de dados
# (Tempo aproximado: 20 minutos)

# executaBaseDeDados()